In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

Use the random seed to reproduce the same result every time if the script is kept consistent otherwise each run will produce different results

In [ ]:
np.random.seed(500)

Set the encoding to ‘latin-1’ because the text has many special characters.

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
Corpus= pd.read_csv(io.BytesIO(uploaded['amazon.csv']),encoding='latin-1')
Corpus.head(2)

Saving amazon.csv to amazon.csv


text     label
0  Stuning even for the non-gamer: This sound tra...  positive
1   The best soundtrack ever to anything.: I'm re...  positive

In [ ]:
# Step - a : Remove blank rows if any.
Corpus['text'].dropna(inplace=True)

In [ ]:
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['text'] = [entry.lower() for entry in Corpus['text']]

In [ ]:
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
nltk.download('punkt')
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
Corpus.head(2)

text     label
0  [stuning, even, for, the, non-gamer, :, this, ...  positive
1  [the, best, soundtrack, ever, to, anything, .,...  positive

In [ ]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
nltk.download('wordnet')
nltk.download('omw-1.4')
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
for index,entry in enumerate(Corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [ ]:
Corpus.head(2)

text     label  \
0  [stuning, even, for, the, non-gamer, :, this, ...  positive   
1  [the, best, soundtrack, ever, to, anything, .,...  positive   

                                          text_final  
0  ['stun', 'even', 'sound', 'track', 'beautiful'...  
1  ['best', 'soundtrack', 'ever', 'anything', 're...

In [ ]:
# Step - 2: Split the model into Train and Test Data set
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

In [ ]:
#Label encode the target variable
#This is done to transform ground truths to numerical values
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
Train_X[:10]

2150    ['read', 'discover', 'need', 'use', 'think', '...
8629    ['pocket', 'book', 'guide', 'lao', 'excellent'...
8703    ['dumb', 'movie', 'big', 'waste', 'time', 'eve...
7060    ['difficult', 'parent', 'quality', 'text', 'go...
2735    ['threesome', 'tank', 'suppose', 'victimize', ...
2597    ['true', 'novel', 'really', 'really', 'awful',...
7206    ['good', 'enjoy', 'read', 'book', 'fireman', '...
3890    ['swing', 'swing', 'swing', 'enjoy', 'diverse'...
1115    ['helpful', 'early', 'pregnancy', 'use', 'pill...
2918    ['meet', 'eye', 'dollar', 'hour', 'never', 'ge...
Name: text_final, dtype: object

In [ ]:
Train_Y[:10]

array([1, 0, 0, 1, 1, 0, 1, 0, 0, 0])

In [ ]:
# Vectorize the words by using TF-IDF Vectorizer
#This is done to find how important a word in document is in comaprison to the corpus
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
print(Tfidf_vect.vocabulary_)

{'stun': 4264, 'even': 1532, 'sound': 4109, 'track': 4552, 'beautiful': 384, 'paint': 3157, 'mind': 2833, 'well': 4868, 'would': 4953, 'recomend': 3588, 'people': 3222, 'hate': 2043, 'video': 4769, 'game': 1861, 'music': 2925, 'play': 3294, 'cross': 1028, 'ever': 1536, 'best': 421, 'back': 330, 'away': 320, 'crude': 1031, 'take': 4365, 'fresh': 1819, 'step': 4203, 'guitar': 1999, 'soulful': 4106, 'orchestra': 3093, 'impress': 2234, 'anyone': 203, 'care': 636, 'listen': 2600, 'soundtrack': 4111, 'anything': 204, 'read': 3554, 'lot': 2644, 'review': 3706, 'say': 3818, 'figure': 1700, 'write': 4959, 'disagree': 1226, 'bit': 442, 'ultimate': 4631, 'masterpiece': 2742, 'timeless': 4500, 'year': 4978, 'beauty': 386, 'simply': 3991, 'refuse': 3611, 'price': 3402, 'tag': 4364, 'pretty': 3397, 'must': 2931, 'go': 1928, 'buy': 588, 'cd': 669, 'much': 2915, 'money': 2872, 'one': 3073, 'feel': 1677, 'worth': 4949, 'every': 1538, 'penny': 3221, 'amaze': 149, 'favorite': 1666, 'time': 4499, 'hand': 

In [ ]:
print(Train_X_Tfidf)

  (0, 4500)	0.37634188677099956
  (0, 4499)	0.1502086671688917
  (0, 3952)	0.35870975205557054
  (0, 3868)	0.25152943577361386
  (0, 3838)	0.2690840463105974
  (0, 3730)	0.3469774999759746
  (0, 3643)	0.28971770688512954
  (0, 3554)	0.29440491517773787
  (0, 2931)	0.22969709983777647
  (0, 1936)	0.13398240399394393
  (0, 1532)	0.17762585383071805
  (0, 513)	0.3210759641783664
  (0, 485)	0.1230432680090133
  (0, 240)	0.24487094004433968
  (1, 4693)	0.2138508178671446
  (1, 4049)	0.356700675875624
  (1, 3428)	0.21279978145402734
  (1, 3309)	0.8157664821375503
  (1, 2581)	0.21719271428576165
  (1, 1248)	0.20747717579157313
  (1, 588)	0.16141344909333402
  (2, 4740)	0.18369761701331289
  (2, 4623)	0.1499525624356807
  (2, 4460)	0.10285168719742008
  (2, 4184)	0.11682860303877614
  :	:
  (6998, 2559)	0.12648150924495427
  (6998, 2508)	0.11841716778929348
  (6998, 2114)	0.14040672300699006
  (6998, 1969)	0.0732676023027426
  (6998, 1785)	0.22643036986644238
  (6998, 1753)	0.2050525986673158


In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  83.3


In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  84.53333333333333
